In [1]:
import xarray as xr
import numpy as np
import os

OUTPUT_LOCATION = "/Odyssey/public/glonet/"


In [2]:
in1 = xr.open_dataset("/Users/woney/Downloads/in1.nc")
# out1 = xr.open_dataset("/Users/woney/Downloads/")


In [3]:
ch_size = in1.sizes["ch"]*in1.sizes["time"]
len(in1.variables)

5

In [4]:
(in1.to_array(dim="variable")
    .stack(newdim=("ch", "time"))
    # .reset_index("newdim", drop=True)
    # .assign_coords(newdim=np.arange(in1.sizes["ch"]*in1.sizes["time"]))
    # .transpose("variable", "lat", "lon", "newdim")
    # .to_dataset(dim="variable"))
)


<xarray.DataArray (variable: 1, lat: 672, lon: 1440, newdim: 10)> Size: 39MB
array([[[[        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         ...,
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan]],

        [[        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
...
           0.0809295 ,  0.04644614],
         [-0.24519701, -0.2484137 , -1.4150205 , ..., -0.04026698,
           0.08105475,  0.04658608],
         [-0.2450979 , -0.24831098, -1.4151456 , ..., -0.0400134 ,
           0.08111223,  0.04667615]],

        [[-0.2488609 , -0.25197637, -1.4147832 , ..., -0.04055484,
           0.07926486,  0.04368005],
         [-0.2488116 , -0.2519263 , -1.4148164 , ..., -0.04034806,
           0.07919464,  0.04367702],
         [-0.24876344, -0.2518774 , -1.4148499 , ..., -0.04014632,
           0.07909936,  0.0436536 ],
         ...,
         [-0.2490137 , -0.25213164, -1.4146875 , ..., -0.0412261 ,
           0.07923056,  0.04349068],
         [-0.24896215, -0.25207925, -1.4147185 , ..., -0.04099579,
           0.0792812 ,  0.04358514],
         [-0.24891116, -0.25202745, -1.4147505 , ..., -0.04077139,
           0.07929318,  0.04364885]]]],
      shape=(1, 672, 1440, 10), dtype=float32)
Coordinates:
  * lon       (lon) float64 12kB -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * lat       (lat) float64 5kB -78.0 -77.75 -77.5 -77.25 ... 89.25 89.5 89.75
  * variable  (variable) object 8B 'data'
  * newdim    (newdim) object 80B MultiIndex
  * ch        (newdim) int32 40B 0 0 1 1 2 2 3 3 4 4
  * time      (newdim) datetime64[ns] 80B 2025-06-30 2025-07-01 ... 2025-07-01

In [ ]:
import xarray as xr
import numpy as np

# Reshape Forecast Dataset to GLONET input (3 files) Dataset shape. 
# i.e. flatten depth dimension and passe it to channel with variables.


def reshapeDataset(in_ds : xr.Dataset) -> xr.Dataset:
        
    out_ds = (in_ds.to_array(dim = "variable")                              # Convert to xarray DatArray
                        .stack(ch=("variable", "depth"))                        # Merge two dimensions one dimension "ch"
                        .reset_index("ch", drop=True)                           # Clear void coordinates 
                        .assign_coords(ch=np.arange(len(in_ds.variables) * in_ds.sizes["depth"])) # define values of ch 
                        .transpose("time", "ch", "latitude", "longitude")       # Reshape in order
                        .rename({"latitude" : "lat", "longitude" : "lon"})      # Rename into input Dataset config
                        .to_dataset)                                            # Reconvert to Dataset
        
    return out_ds

# Divide forcast NetCDF file in to 3 files by states depth
def divideIntoThree(input_nc : str, 
                    cycle : int) :
    
    # Extract date for output name
    forecast_date = input_nc["time"].values[cycle - 1]
    
    # Read netCDF file and pick only two last timeset
    last_two_day_forecast = xr.open_dataset(input_nc).isel(time = [cycle-2, cycle-1])
    
    # Slice datatset by depth first 
    surface = last_two_day_forecast.isel(depth = 0, missing_dims="ignore")
    deep = last_two_day_forecast.sel(depth = slice(40, 800))
    deeper = last_two_day_forecast.sel(depth = slice(900, 5500))
    
    # Make a dictionary for iterattion
    ds_map = {
            "1" : surface,
            "2" : deep,
            "3" : deeper
    }
    
    # Write input file
    out_dir = OUTPUT_LOCATION + "/init??"
    os.makedirs(out_dir, exist_ok=True)
    
    in123 = []
    
    for i in ds_map :
        reshapeDataset(i).to_netcdf(f"{out_dir}/{forecast_date}_autoregress_init_states")
        in123.append(reshapeDataset(i))
        
    return in123
